# 三大法人期貨淨未平倉量口數  
* 抓取目標包含大台、小台、微型台指期貨，以及金融期貨、電子期貨  
* 三大法人（外資、投信、自營商）期貨未平倉口數淨額  
* 使用 finlab api 取得資料  

In [1]:
import os
import pandas as pd
import requests
import duckdb

In [2]:
from finlab import data

In [3]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings
from proj_util_pkg.finlab_api import finlab_manager as flm

## 公用參數設定

In [4]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [5]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

輸入成功!


## 外部資料讀取  

In [ ]:
# 外資期貨未平倉口數淨額
net_oi = data.get('futures_institutional_investors_trading_summary:多空未平倉口數淨額', save_to_storage=True)

In [7]:
# 選取特定欄位
target_cols = [
    "臺股期貨_外資及陸資", "臺股期貨_投信", "臺股期貨_自營商",  
    "小型臺指期貨_外資及陸資", "小型臺指期貨_投信", "小型臺指期貨_自營商",
    "微型臺指期貨_外資及陸資", "微型臺指期貨_投信", "微型臺指期貨_自營商",
    "金融期貨_外資及陸資", "金融期貨_投信", "金融期貨_自營商",
    "電子期貨_外資及陸資", "電子期貨_投信", "電子期貨_自營商"
]

net_oi_df = net_oi.loc[:, target_cols]
net_oi_df.reset_index(inplace=True)

# column name 含有 "外資及陸資" 的欄位，將其名稱改為 "外資"
net_oi_df.columns = net_oi_df.columns.str.replace("外資及陸資", "外資")

# 將 date 欄位 rename 為"Date" 欄位轉換為 datetime 格式
net_oi_df.rename(columns={"date": "Date"}, inplace=True)
net_oi_df["Date"] = pd.to_datetime(net_oi_df["Date"])

# 取近2個月資料
net_oi_df = net_oi_df[net_oi_df["Date"] >= pd.Timestamp.now() - pd.DateOffset(months=2)]

print(net_oi_df.shape)
net_oi_df.tail(3)


(42, 16)


symbol,Date,臺股期貨_外資,臺股期貨_投信,臺股期貨_自營商,小型臺指期貨_外資,小型臺指期貨_投信,小型臺指期貨_自營商,微型臺指期貨_外資,微型臺指期貨_投信,微型臺指期貨_自營商,金融期貨_外資,金融期貨_投信,金融期貨_自營商,電子期貨_外資,電子期貨_投信,電子期貨_自營商
4246,2024-09-30,-37202.0,24731.0,-824.0,2222.0,128.0,-9273.0,-7870.0,0.0,-5402.0,-414.0,11.0,300.0,-353.0,199.0,2.0
4247,2024-10-01,-39479.0,24890.0,-759.0,3561.0,125.0,-8483.0,-6251.0,0.0,-3399.0,-424.0,11.0,306.0,-393.0,198.0,34.0
4248,2024-10-04,-37000.0,24631.0,-1131.0,5893.0,124.0,-7678.0,-5536.0,0.0,-2395.0,-421.0,11.0,307.0,-428.0,208.0,65.0


## 資料留存ＤＢ

In [8]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [9]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [10]:
table_name = "tw_futures_institutional_investors_trading_summary"

In [11]:
# 針對DataFrame，以Date單筆先確認表中，欄位Date沒有重複資料時，才進行單筆insert
for i in range(len(net_oi_df)):
    try:
        net_oi_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)
    except Exception as e:
        # print(e)
        pass


/var/folders/14/b8c6vz6n79n4wt12qny6djk40000gn/T/ipykernel_46308/1382886147.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  net_oi_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)


In [12]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc").fetch_df()

,Date,臺股期貨_外資,臺股期貨_投信,臺股期貨_自營商,小型臺指期貨_外資,小型臺指期貨_投信,小型臺指期貨_自營商,金融期貨_外資,金融期貨_投信,金融期貨_自營商,電子期貨_外資,電子期貨_投信,電子期貨_自營商,微型臺指期貨_外資,微型臺指期貨_投信,微型臺指期貨_自營商
0,2024-10-04,-37000,24631,-1131,5893,124,-7678,-421,11,307,-428,208,65,-5536.0,0.0,-2395.0
1,2024-10-01,-39479,24890,-759,3561,125,-8483,-424,11,306,-393,198,34,-6251.0,0.0,-3399.0
2,2024-09-30,-37202,24731,-824,2222,128,-9273,-414,11,300,-353,199,2,-7870.0,0.0,-5402.0
3,2024-09-27,-36237,25284,-1408,1270,134,-7849,-454,11,313,-294,199,-21,-3410.0,0.0,-829.0
4,2024-09-26,-37081,25632,-1532,3626,134,-7059,-421,11,316,-360,199,7,-224.0,0.0,1871.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2024-04-23,-12505,10709,-5069,8906,271,-16307,-196,7,-297,-15,195,-400,NaN,NaN,NaN
112,2024-04-22,-16400,10164,-2855,665,294,-15411,-196,7,-258,-53,190,-339,NaN,NaN,NaN
113,2024-04-19,-19777,9799,-1438,-4007,336,-15259,-375,7,-102,-38,182,-296,NaN,NaN,NaN
114,2024-04-18,-14700,9114,-2328,6518,346,-8875,-364,7,-139,78,176,-336,NaN,NaN,NaN


In [13]:
# 關閉資料庫連線
conn_duckdb.close()